In [0]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("sep13").getOrCreate()
dbutils.fs.cp("File:/Workspace/Shared/sales_data.csv","dbfs:/FileStore/sales_data.csv")
dbutils.fs.cp("File:/Workspace/Shared/customer_data.json","dbfs:/FileStore/customer_data.json")
dbutils.fs.cp("File:/Workspace/Shared/new_sales_data.csv","dbfs:/FileStore/new_sales_data.csv")


True

In [0]:
df_sales_data=spark.read.format("csv").option("header","true").load("dbfs:/FileStore/sales_data.csv")
df_sales_data.write.format("delta").mode("overwrite").save("dbfs:/FileStore/delta/delta_sales_data")

In [0]:
df_customer_data=spark.read.format("json").option("multiline","true").load("dbfs:/FileStore/customer_data.json")
df_customer_data.write.format("delta").mode("overwrite").save("dbfs:/FileStore/delta/delta_customer_data")


In [0]:
df_new_sales_data=spark.read.format("csv").option("header","true").load("dbfs:/FileStore/new_sales_data.csv")
df_new_sales_data.write.format("delta").mode("overwrite").save("dbfs:/FileStore/delta/delta_new_sales_data")

In [0]:
df_sales_data=spark.read.format("delta").load("dbfs:/FileStore/delta/delta_sales_data")
df_new_sales_data=spark.read.format("delta").load("dbfs:/FileStore/delta/delta_new_sales_data")

In [0]:
df_sales_data.createOrReplaceTempView("delta_sales_data") 
df_new_sales_data.createOrReplaceTempView("delta_new_sales_data")

spark.sql
("""
 MERGE INTO delta_sales_data AS target
USING delta_new_sales_data AS source
ON target.OrderID = source.OrderID
WHEN MATCHED THEN UPDATE SET
    target.Quantity = source.Quantity
WHEN NOT MATCHED THEN INSERT
(
    OrderID, OrderDate, CustomerID, Product, Quantity, Price
)
VALUES (
    source.OrderID, source.OrderDate, source.CustomerID, source.Product, source.Quantity, source.Price
)
""")

'\n MERGE INTO delta_sales_data AS target\nUSING delta_new_sales_data AS source\nON target.OrderID = source.OrderID\nWHEN MATCHED THEN UPDATE SET\n    target.Quantity = source.Quantity\nWHEN NOT MATCHED THEN INSERT\n(\n    OrderID, OrderDate, CustomerID, Product, Quantity, Price\n)\nVALUES (\n    source.OrderID, source.OrderDate, source.CustomerID, source.Product, source.Quantity, source.Price\n)\n'

In [0]:
spark.sql("select * from delta_sales_data") .show()  
spark.sql("describe delta_sales_data").show()
spark.sql("create table if not exists delta_sales_table as select * from delta_sales_data")
spark.sql("select * from delta_sales_table").show()
spark.sql("describe delta_sales_table").show()
spark.sql("OPTIMIZE delta_sales_table ZORDER BY (OrderID)")

+-------+----------+----------+--------+--------+-----+
|OrderID| OrderDate|CustomerID| Product|Quantity|Price|
+-------+----------+----------+--------+--------+-----+
|   1001|15-01-2024|      C001|Widget A|      10| 25.5|
|   1003|16-01-2024|      C001|Widget C|       8| 22.5|
|   1004|17-01-2024|      C003|Widget A|      15| 25.5|
|   1005|18-01-2024|      C004|Widget D|       7|   30|
|   1006|19-01-2024|      C002|Widget B|       9|15.75|
|   1007|20-01-2024|      C005|Widget C|      12| 22.5|
|   1008|21-01-2024|      C003|Widget A|      10| 25.5|
|   1002|16-01-2024|      C002|Widget B|      10|15.75|
|   1009|22-01-2024|      C006|Widget E|      14|   20|
|   1010|23-01-2024|      C007|Widget F|       6|   35|
+-------+----------+----------+--------+--------+-----+

+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|   OrderID|   string|   NULL|
| OrderDate|   string|   NULL|
|CustomerID|   string|   NULL|
|   Product|   string|   NULL

DataFrame[path: string, metrics: struct<numFilesAdded:bigint,numFilesRemoved:bigint,filesAdded:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,filesRemoved:struct<min:bigint,max:bigint,avg:double,totalFiles:bigint,totalSize:bigint>,partitionsOptimized:bigint,zOrderStats:struct<strategyName:string,inputCubeFiles:struct<num:bigint,size:bigint>,inputOtherFiles:struct<num:bigint,size:bigint>,inputNumCubes:bigint,mergedFiles:struct<num:bigint,size:bigint>,numOutputCubes:bigint,mergedNumCubes:bigint>,clusteringStats:struct<inputZCubeFiles:struct<numFiles:bigint,size:bigint>,inputOtherFiles:struct<numFiles:bigint,size:bigint>,inputNumZCubes:bigint,mergedFiles:struct<numFiles:bigint,size:bigint>,numOutputZCubes:bigint>,numBins:bigint,numBatches:bigint,totalConsideredFiles:bigint,totalFilesSkipped:bigint,preserveInsertionOrder:boolean,numFilesSkippedToReduceWriteAmplification:bigint,numBytesSkippedToReduceWriteAmplification:bigint,startTimeMs:bigint,endTimeMs:bigint,

In [0]:
spark.sql("Describe History delta_sales_table").show(truncate=False)

+-------+-------------------+----------------+----------------------------------+----------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+----+------------------+--------------------+-----------+-----------------+-------------+------------------------------------------------------------+------------+------------------------------------------+
|version|timestamp          |userId          |userName                          |operation             |operationParameters                                                                                                                                     |job |notebook          |clusterId           |readVersion|isolationLevel   |isBlindAppend|operationMetrics                                            |userMetadata|engineInfo                                |
+-------+-------------------+----------------+--------------------------

In [0]:
spark.sql("vacuum delta_sales_table retain 170  hours")

DataFrame[path: string]

In [0]:
spark.sql("RESTORE TABLE delta_sales_table TO VERSION AS OF 2;")

DataFrame[]

In [0]:
%sql
create table managed_tabel(
  id INT,
  name String
);

In [0]:
%sql
create external table external_table(
  id INT,
  name string
)
Location '/user/data/external_data';